# Задача 8
### Плюснин Павел 578 / plyusnin.pa@phystech.edu
Сначала посчитаем BLEU-скоры для всех пар переводчиков. Занесем эти данные в словарь bleus

In [44]:
transs = {}
bleus = {}

In [45]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
import glob
import re

for file_name in glob.glob("./*.txt"):
    with open(file_name, encoding="utf8") as file:
        lang_from, lang_to, ttype = re.findall(".*(..)_(..)_(.).*", file_name)[0]
        text = file.read()
        text = tokenizer.tokenize(text)
        try:
            transs[lang_from][lang_to][ttype] = list(map(lambda x: x.lower(), text))
        except KeyError:
            try:
                transs[lang_from][lang_to] = dict()
                transs[lang_from][lang_to][ttype] = list(map(lambda x: x.lower(), text))
                
                bleus[lang_from][lang_to] = dict()
            except KeyError:
                transs[lang_from] = dict()
                transs[lang_from][lang_to] = dict()
                transs[lang_from][lang_to][ttype] = list(map(lambda x: x.lower(), text))
                
                bleus[lang_from] = dict()
                bleus[lang_from][lang_to] = dict()

In [3]:
from nltk.translate.bleu_score import sentence_bleu
for from_lang, dictt in transs.items():
    for to_lang, dictt in dictt.items():
        for ttype, text in dictt.items():
            if ttype != "g":
                bleus[from_lang][to_lang][ttype] = sentence_bleu([transs[from_lang][to_lang]["g"]], text)

In [4]:
bleus

{'en': {'de': {'n': 0.1990509245610674,
   'p': 0.19103087482685588,
   'z': 0.10009487408556153},
  'ru': {'n': 0.17524305655225547,
   'p': 0.22330156919355987,
   'z': 0.14029326439049417}},
 'kk': {'ru': {'n': 0.3602675096515449,
   'p': 0.2675619204061147,
   'z': 0.3606710891026718}}}

Требуется определить для каких пар языков можно выделить какого-либо лидера среди переводчиков

Требуется как-то формализовать нашу задачу. Будем считать для каждой пары языков, что $X_1$ - выборка из двух скоров, не являющимися макс. элементами, а $X_2$ - выборка, сотсоящая как раз из нашего лидера:  
$$X_1 = (s_{(1)}, s_{(2)}), X_2 = (s_{(3)})$$  
$$H_0: \mathbb{E}X_1 = \mathbb{E}X_2: \text{среднее не изменилось}$$
$$H_1: \mathbb{E}X_1 \neq \mathbb{E}X_2: \text{среднее увеличилось}$$
Стоит отметить, что гипотезы выше сформулированы неформально, конкретные гипотезы зависят от используемого критерия.  
Но с такой формализацией мы терпим фиаско, так как ни один критерий не может справиться с такими малыми выборками размера 1 и 2:

In [5]:
import scipy.stats as st

In [6]:
pairs = {}
for from_lang, dictt in bleus.items():
    for to_lang, dictt in dictt.items():
        pair = from_lang + "-" + to_lang
        scores = dictt.values()
        maxx = max(scores)
        pairs[pair] = [[score for score in scores if score != maxx], [maxx]]

In [7]:
pairs

{'en-de': [[0.19103087482685588, 0.10009487408556153], [0.1990509245610674]],
 'en-ru': [[0.17524305655225547, 0.14029326439049417], [0.22330156919355987]],
 'kk-ru': [[0.3602675096515449, 0.2675619204061147], [0.3606710891026718]]}

Критерии не применяются в связи с малостью выборок. Ни о каких поправках на множественность гипотез, мы тут не говорим)

In [8]:
for name, vals in pairs.items():
    p_val = st.mannwhitneyu(vals[0], vals[1], alternative= 'less')
    print(name, p_val)

en-de MannwhitneyuResult(statistic=0.0, pvalue=0.27014568730370997)
en-ru MannwhitneyuResult(statistic=0.0, pvalue=0.27014568730370997)
kk-ru MannwhitneyuResult(statistic=0.0, pvalue=0.27014568730370997)


In [9]:
from permute.core import two_sample
for name, vals in pairs.items():
    p_val = two_sample(vals[0], vals[1],  alternative='less') 
    print(name, p_val)

en-de (0.0, -0.053488050104858714)
en-ru (0.0, -0.06553340872218505)
kk-ru (0.0, -0.04675637407384198)


Переформализуем задачу:
Будем считать для каждой пары языков, что $X$ - выборка из двух разниц максимального скора и одного из двух не max скора:
$$X = (X_{(3)} - X_{(1)}, X_{(3)} - X_{(2)})$$
Для полученной выборки будем оценивать, насколько ее среднее отличимо от 0. Стоит заметить, что неформально мы тут проверяем скорее не наличие лидера, а наличие проигравших, но это не важно ;)

In [10]:
deltas = {}
for from_lang, dictt in bleus.items():
    for to_lang, dictt in dictt.items():
        pair = from_lang + "-" + to_lang
        scores = dictt.values()
        maxx = max(scores)
        deltas[pair] = [maxx - score for score in scores if score != maxx]

In [11]:
deltas

{'en-de': [0.008020049734211526, 0.09895605047550587],
 'en-ru': [0.048058512641304396, 0.0830083048030657],
 'kk-ru': [0.00040357945112690086, 0.09310916869655711]}

In [12]:
for name, vals in deltas.items():
    p_val = st.ttest_1samp(vals, 0)
    print(name, p_val)

en-de Ttest_1sampResult(statistic=1.1763888816053816, pvalue=0.4485166697473929)
en-ru Ttest_1sampResult(statistic=3.7501458331351905, pvalue=0.1658984718678795)
kk-ru Ttest_1sampResult(statistic=1.0087066908136133, pvalue=0.4972406041971509)


По этому тесту Нулевую гипотезу отвергнуть нельзя. Оценим дисперсию нашей выборки, выведем интервалы $[\mu - 2\sigma; \mu + 2\sigma]$

In [13]:
import numpy as np
for name, vals in deltas.items():
    sigma = np.std(vals, ddof=1) #при ddof мы также отвергаем гипотезу для 2ой пары
    print(name, (np.mean(vals) - 2 *sigma, np.mean(vals) + 2 *sigma))

en-de (-0.07511487545144958, 0.182090975661167)
en-ru (0.016106938644901314, 0.11495987879946878)
kk-ru (-0.08434912754483473, 0.17786187569251874)


Ура! для пары en-ru доверительный интервал не включает 0, поэтому по правилу $2\sigma$ мы можем отвергнуть гипотезу об отсутсвии лидера! (на уровне значимости $\approx 0.05$)

Применим поправки на множественность гипотез. Так как у нас вприницпе единственный кандидат на отвержение, то разницы между методом Бонферонни и методом Холма у нас нет.

Поищем соответствующее количество сигм для соответствующей $\alpha / 3$

In [14]:
import scipy.integrate as integrate
n_sigmas = np.linspace(2, 2.5)
for sigm in n_sigmas:
    print(sigm, (0.05/3) - (1- integrate.quad(st.norm.pdf, -sigm, sigm)[0]))

2.0 -0.02883359722969164
2.010204081632653 -0.02774292693709651
2.020408163265306 -0.02667440070485255
2.0306122448979593 -0.02562767793135839
2.0408163265306123 -0.02460242047114874
2.0510204081632653 -0.023598292689186947
2.061224489795918 -0.02261496151301084
2.0714285714285716 -0.02165209648274685
2.0816326530612246 -0.020709369799011997
2.0918367346938775 -0.0197864563687225
2.1020408163265305 -0.018883033848828886
2.1122448979591835 -0.017998782687998646
2.122448979591837 -0.01713338616626908
2.13265306122449 -0.0162865304326914
2.142857142857143 -0.015457904540989994
2.1530612244897958 -0.014647200483261565
2.163265306122449 -0.013854113221738403
2.173469387755102 -0.013078340718641498
2.183673469387755 -0.012319583964147503
2.193877551020408 -0.011577547002499963
2.204081632653061 -0.010851936956289217
2.2142857142857144 -0.01014246404892932
2.2244897959183674 -0.009448841625359469
2.2346938775510203 -0.008770786171002739
2.2448979591836733 -0.008108017329003844
2.2551020408163

In [15]:
n_sigmas = np.linspace(2.387755, 2.397959)
for sigm in n_sigmas:
    print(sigm, (0.05/3) - (1- integrate.quad(st.norm.pdf, -sigm, sigm)[0]))

2.387755 -0.0002849715534713436
2.387963244897959 -0.00027536939568478455
2.3881714897959183 -0.0002657720116830374
2.3883797346938773 -0.00025617939950911203
2.3885879795918368 -0.00024659155720624035
2.388796224489796 -0.00023700848281809836
2.389004469387755 -0.00022743017438936128
2.3892127142857142 -0.00021785662996448227
2.3894209591836733 -0.00020828784758913574
2.3896292040816327 -0.00019872382530921814
2.3898374489795917 -0.00018916456117184718
2.3900456938775507 -0.00017961005322314136
2.39025393877551 -0.00017006029951199472
2.390462183673469 -0.000160515298085636
2.3906704285714286 -0.00015097504699384742
2.3908786734693876 -0.00014143954428485692
2.391086918367347 -0.00013190878800966796
2.391295163265306 -0.00012238277621828483
2.391503408163265 -0.00011286150696171102
2.3917116530612246 -0.00010334497829172715
2.3919198979591836 -9.383318826077999e-05
2.392128142857143 -8.432613492087224e-05
2.392336387755102 -7.482381632589394e-05
2.392544632653061 -6.53262305299572e-05

In [16]:
for name, vals in deltas.items():
    sigma = np.std(vals, ddof=1) 
    print(name, (np.mean(vals) - 2.394 *sigma, np.mean(vals) + 2.394 *sigma))

en-de (-0.10044965178604233, 0.20742575199575974)
en-ru (0.006369924039676414, 0.12469689340469368)
kk-ru (-0.11017691136371405, 0.20368965951139806)


Мы все так же отвергаем вторую гипотезу даже с поправкой! Таким образом для $en-ru$ мы действительно отвергаем гипотезу об отсутсвии проигравших, т.е лидера **можно выделить**!

Стоит отметить, что мы предположили нормальность распредления наших дельт, а также в связи с малостью выборок наши оценки все же не очень надежны

# UPDATE

In [16]:
transs = {}
bleus = {}

In [17]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
import glob
import re

for file_name in glob.glob("./*.txt"):
    with open(file_name, encoding="utf8") as file:
        lang_from, lang_to, ttype = re.findall(".*(..)_(..)_(.).*", file_name)[0]
        
        try:
            transs[lang_from][lang_to][ttype]
        except KeyError:
            try:
                transs[lang_from][lang_to][ttype] = list()
            except KeyError:
                try:
                    transs[lang_from][lang_to] = dict()
                    transs[lang_from][lang_to][ttype] = list()
                    
                    bleus[lang_from][lang_to] = dict()
                except KeyError:
                    transs[lang_from] = dict()
                    transs[lang_from][lang_to] = dict()
                    transs[lang_from][lang_to][ttype] = list()
                    
                    bleus[lang_from] = dict()
                    bleus[lang_from][lang_to] = dict()
            
        for text in file:
            text = tokenizer.tokenize(text)
            transs[lang_from][lang_to][ttype].append(list(map(lambda x: x.lower(), text)))

In [20]:
from nltk.translate.bleu_score import sentence_bleu
for from_lang, dictt in transs.items():
    for to_lang, dictt in dictt.items():
        for ttype, text in dictt.items():
            if ttype != "g":
                bleus[from_lang][to_lang][ttype] = list()
                for index, sentence in enumerate(text):
                    bleus[from_lang][to_lang][ttype].append(sentence_bleu([transs[from_lang][to_lang]["g"][index]], sentence))

C:\Users\plyus\AppData\Local\conda\conda\envs\py36\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
C:\Users\plyus\AppData\Local\conda\conda\envs\py36\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
C:\Users\plyus\AppData\Local\conda\conda\envs\py36\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [21]:
bleus

{'en': {'de': {'n': [0.4293663233009672,
    0.20591724619179663,
    0.4734490597792576,
    0.2812630724452771,
    0.2646625485422896,
    0.13120966411271084,
    0.19940445989088912,
    0.5623413251903491,
    0.37239098949398236,
    0.34295479618200586],
   'p': [0.31007120066002053,
    0.16830297866366878,
    0.5196599018161566,
    0.3800427070295605,
    0.5814307369682193,
    0.1318331306548015,
    0.170595737016168,
    0.2559142512628947,
    0.6076795808137692,
    0.22878386498145054],
   'z': [0.17694975149532557,
    0.5755901146501323,
    0.23819486101149287,
    0.36209781491447873,
    0.7162326270441588,
    0.1312155138646134,
    0.4851178848133934,
    0.7259795291154771,
    0.3508439695638686,
    0.32865231624539687]},
  'ru': {'n': [0.44116293593227063,
    0.3148282689155186,
    0.3040559696901293,
    0.29693621041385787,
    0.14440028187544326,
    0.32555630133216146,
    0.5900468726392808,
    0.360056585428503,
    0.6334717766551771,
    0.66

In [41]:
import scipy.stats as st

In [52]:
import numpy as np
from permute.core import one_sample

In [77]:
from statsmodels.stats.descriptivestats import sign_test

In [81]:
deltas = {}
combos = ((0,1), (1,0), (1,2), (2,1), (0,2), (2,0))
for from_lang, dictt in bleus.items():
    for to_lang, dictt in dictt.items():
        pair = from_lang + "-" + to_lang
        scores = list(dictt.values())
        p_vals_wilcoxon = []
        p_vals_permute = []
        p_vals_sign = []
        p_vals_ttest = []
        for index, combo in enumerate(combos):
            p_vals_wilcoxon.append(st.wilcoxon(scores[combo[0]], scores[combo[1]]))
            p_vals_permute.append(two_sample(np.array(scores[combo[0]]), np.array(scores[combo[1]]), alternative="less"))
            p_vals_sign.append(sign_test(np.array(scores[combo[0]]) - np.array(scores[combo[1]])))
            p_vals_ttest.append(st.ttest_rel(np.array(scores[combo[0]]), np.array(scores[combo[1]])))
            
            #мы проверяем по 3 пары для 3 пар языков
            if (p_vals_wilcoxon[index][1] < (0.1 / (3*3))):
                print(combo, "declined by Wilcoxon for", pair)
            if (p_vals_permute[index][0] < 0.1 / (3*3)): 
                print(combo, "declined by PermuteTest for", pair)
            if (p_vals_sign[index][1] < 0.1 / (3*3)): 
                print(combo, "declined by SignTest for", pair)
            if (p_vals_ttest[index][1] < 0.1 / (3*3)): 
                print(combo, "declined by ttest for", pair)

C:\Users\plyus\AppData\Local\conda\conda\envs\py36\lib\site-packages\scipy\stats\morestats.py:2385: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


Как мы видим, даже на уровне значимости 0.1 с поправкой Бонферонни на 9 гипотез (что среди 3 пар языков есть лидирующий переводчик в 3 парах переводчиков) нельзя выделить лидера. Итоговый ответ смотри выше, до UPDATE блока